#### Задание
1. Взять предобученную трансформерную архитектуру и решить задачу перевода (для тогоже корпуса что вы выбрали из предыдущего дз)

In [12]:
import pandas as pd
import numpy as np

In [2]:
from transformers import FSMTForConditionalGeneration, FSMTTokenizer
mname = "facebook/wmt19-en-ru"
tokenizer = FSMTTokenizer.from_pretrained(mname)
model = FSMTForConditionalGeneration.from_pretrained(mname)

In [1]:
from datasets import load_dataset

dataset = load_dataset("cosmos_qa")

Using custom data configuration default


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/25262 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6963 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2985 [00:00<?, ? examples/s]

Dataset cosmos_qa downloaded and prepared to /Users/maximdoroshenko/.cache/huggingface/datasets/cosmos_qa/default/0.1.0/e4e873eafb86b174fbfdf309a8baac27525753e655fe93f48f99023efa950ae6. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
df = pd.DataFrame(dataset["train"])

In [8]:
df.head()

,id,context,question,answer0,answer1,answer2,answer3,label
0,3Q9SPIIRWJKVQ8244310E8TUS6YWAC##34V1S5K3GTZMDU...,Good Old War and person L : I saw both of thes...,"In the future , will this person go to see oth...",None of the above choices .,This person likes music and likes to see the s...,This person only likes Good Old War and Person...,Other Bands is not on tour and this person can...,1
1,3E24UO25QZOMYXHZN4TEH9EMT9GO6L##3UN61F00HXNWYQ...,I mean it : not one person said ANYTHING to me...,Why might have the temp agency tell me I am no...,The company hiring the temp workers might have...,The temp agency hiring the temp workers might ...,None of the above choices .,I might have had a change of mind working for ...,0
2,3M4KL7H8KVL125AYH2V35D1E0A016T##3SB5N7Y3O426ET...,Leaving my shift Thursday day shift I arrived ...,What may have caused the radio to erupt with d...,My partner needed a medic unit .,Someone was running from the ambulances after ...,None of the above choices .,Someone was running from the cops and got into...,3
3,3D5G8J4N5CI2K40F4RZLF9OG2L4VTJ##34MAJL3QP721EU...,"So , last day in Seattle , and my flight was a...",Why did I chit chat with my old manager ?,Because my flight was at 1:30 .,Because I left Seattle feeling really good and...,Because it 's my last day in Seattle .,Because I enjoy talking to him .,3
4,3MQKOF1EE428I44N8B42W7P86DKDW4##3NJM2BJS4ZLBGN...,Ms. Mumma informed Mr. Dail that evidence had ...,Why did I burst into tears ?,Because Ms. Mumma was informative .,Because I had an intense emotional response to...,Because Mr. Dail was informed .,Because I fell out of my chair .,1


In [9]:
def translate(text):
    input_ids = tokenizer.encode(text, return_tensors="pt")
    outputs = model.generate(input_ids)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded

In [16]:
#df['context_ru'] = df['context'].apply(np.vectorize(translate))

In [20]:
df_slice = df.iloc[:10, :]

In [29]:
df_slice['context_ru'] = df_slice['context'].apply(np.vectorize(translate))

/Users/maximdoroshenko/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [36]:
for _, row in df_slice.iterrows():
    print(f"ENG: {row['context']}")
    print(f"RUS: {row['context_ru']}")
    print('\n')

ENG: Good Old War and person L : I saw both of these bands Wednesday night , and they both blew me away . seriously . Good Old War is acoustic and makes me smile . I really can not help but be happy when I listen to them ; I think it 's the fact that they seemed so happy themselves when they played .
RUS: Good Old War и человек L: Я видел обе эти группы в среду вечером, и обе они взорвали меня. Серьезно. Good Old War акустическая и заставляет меня улыбаться.


ENG: I mean it : not one person said ANYTHING to me . They would have if I did something wrong , right ? But , this morning , I got a call from my guy at the temp agency and he said that I was no longer needed in that position , that I did n't need to go out there .
RUS: Но сегодня утром мне позвонил мой парень из агентства Temp, и он сказал, что я больше не нужен на этой должности, что мне не нужно туда ходить.


ENG: Leaving my shift Thursday day shift I arrived the same time as my partner just after six that evening and before

### Вывод:
Перевод удовлетворительный, улавливается контекст. Судя по всему модель 19-го года. Google Translate сейчас переводит интереснее. Например:

ENG:
Leaving my shift Thursday day shift I arrived the same time as my partner just after six that evening and before long the radio erupted in dispatch tones . A car fleeing the police has crashed and landed on its roof with four separate people entrapped inside .

GOOGLE:
Покидая свою смену в четверг, я прибыл в то же время, что и мой напарник, сразу после шести вечера, и вскоре радио взорвалось сигналами депеши. Автомобиль, убегающий от полиции, разбился и приземлился на крышу, внутри оказались зажатыми четыре человека.

MODEL:
Выйдя со своей смены в четверг днем, я прибыл в то же время, что и мой напарник, только после шести вечера, и вскоре радио взорвалось в тонах. Машина, убегавшая от полиции, разбилась и приземлилась на крышу, в которой находились четыре человека.

Ключевая разница: зажатыми и находились. В первом варианте более точно.

#### 2. скачиваем готовый новостной датасет
!wget https://github.com/ods-ai-ml4sg/proj_news_viz/releases/download/data/gazeta.csv.gz
        
        ```
        # пример работы с ним 
        from corus import load_ods_gazeta
        path = 'gazeta.csv.gz'
        records = load_ods_gazeta(path)
        next(records)
        ```

    реализовать метод поиска ближайших статей
    (на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру топ 5-ть или 3-ри, ваш метод должен возвращать топ-k ближайших статей к этому запросу)
    визуально оценить качество

In [39]:
!wget -O gazeta.csv.gz -qq --no-check-certificate https://github.com/ods-ai-ml4sg/proj_news_viz/releases/download/data/gazeta.csv.gz

In [42]:
from corus import load_ods_gazeta
path = 'gazeta.csv.gz'
records = load_ods_gazeta(path)
next(records)

NewsRecord(
    timestamp=datetime.datetime(2008, 11, 21, 15, 19, 14),
    url='https://www.gazeta.ru/news/business/2008/11/21/n_1298950.shtml',
    edition=None,
    topics='Бизнес',
    authors=None,
    title='Госдума сокращает срок действия ставки экспортных пошлин на нефть',
    text='Госдума приняла сегодня в первом чтении и сразу в целом поправки в закон «О таможенном тарифе», сокращающие срок действия ставки экспортных пошлин на нефть с 2-х до 1-го месяца.nnДля установления средних цен на нефть марки Urals и расчета экспортных пошлин правительство России в течение двух месяцев проводит мониторинг на международных рынках нефтяного сырья (средиземноморском и роттердамском), экспортные пошлины на нефть устанавливаются также раз в два месяца.nnСокращение на месяц периода мониторинга (с 15-го числа каждого календарного месяца по 14-е число следующего месяца) и соответственно срока действия ставок экспортных пошлин «позволит более оперативно реагировать на изменения экономической сит

In [48]:
df = pd.DataFrame(records)

In [49]:
df.head()

,0,1,2,3,4,5,6,7
0,2012-09-26 13:54:13,https://www.gazeta.ru/politics/news/2012/09/26...,None,Политика,None,Гордума Новочеркасска приняла отставку мэра го...,В Ростовской области депутаты гордумы Новочерк...,"Stats(fb=None, vk=None, ok=None, twitter=None,..."
1,2012-09-26 21:11:33,https://www.gazeta.ru/culture/news/2012/09/26/...,None,Культура,None,"Министерство культуры назвало фильмы, на котор...",Министерство культуры России определило перече...,"Stats(fb=None, vk=None, ok=None, twitter=None,..."
2,2011-02-24 16:40:29,https://www.gazeta.ru/news/sport/2011/02/24/n_...,None,Спорт,None,Иванов возглавил совет ПБК ЦСКА,Заместитель председателяnправительства РФnСерг...,"Stats(fb=None, vk=None, ok=None, twitter=None,..."
3,2012-09-27 10:32:37,https://www.gazeta.ru/culture/news/2012/09/27/...,None,Культура,None,Коллегия Минкультуры обсудила отказ от бессроч...,Отказ от бессрочных контактов с артистами и вв...,"Stats(fb=None, vk=None, ok=None, twitter=None,..."
4,2012-09-27 10:36:58,https://www.gazeta.ru/culture/news/2012/09/27/...,None,Культура,None,Театр «Россия» на Пушкинской площади открылся ...,Мэр Москвы Сергей Собянин в среду официально о...,"Stats(fb=None, vk=None, ok=None, twitter=None,..."


In [50]:
df.shape

(865843, 8)

https://huggingface.co/symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli

https://www.sbert.net/docs/usage/semantic_textual_similarity.html

In [53]:
# !pip install -U sentence-transformers

In [65]:
from sentence_transformers import SentenceTransformer, util

In [59]:
model = SentenceTransformer('symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli')

In [123]:
sentences = df[6][:1000] # 1000 sentences

In [124]:
embeddings = model.encode(sentences, convert_to_tensor=True)

In [145]:
def get_similar(embeddings, sentence, n):
    """Get <n> similar embeddings for <sentence> and given embeddings"""
    
    assert n > 0, "Bad n"
    
    # Compute embeddings
    s_emb =  model.encode([sentence], convert_to_tensor=True)

    # Compute cosine-similarities for each sentence
    cosine_scores = util.cos_sim(s_emb, embeddings)
    
    # Compute max n scores indexes
    max_scores_idx = np.argpartition(cosine_scores[0], -n)[-n:]
    
    return cosine_scores, max_scores_idx

In [146]:
rnd = np.random.randint(1000, df.shape[0])
sentence = df.iloc[rnd, 6]

sentence

'Российская теннисистка Алиса Клейбанова перенесла операцию на плече.n25-летняя спортсменка пропустит четыре месяца.nКлейбанова будет восстанавливаться от травмы в США.nДругие новости, материалы и статистику можно посмотреть наnстраницеnтенниса.nKleybanova who recovered from cancer in 2011 has flown out to Arizona to work with physio Todd Ellenbecker, a renown shoulder expert'

In [147]:
cosine_scores, max_scores_idx = get_similar(embeddings, sentence, 3)

In [148]:
cosine_scores[0][max_scores_idx]

tensor([0.2586, 0.3722, 0.2685])

In [149]:
for t in sentences[max_scores_idx.tolist()]:
    print(t)
    print("\n")

Начальник войск противовоздушной обороны Сухопутных войск России генерал-лейтенант Александр Леонов сообщил, что создается новый переносной зенитно-ракетный комплекс, который способен преодолевать помехи самых современных лазерных средств противника, сообщаетn«Русская служба новостей»n.nОн добавил, что ПЗРК будет принят на вооружение в течение ближайших лет.n«Создан задел для создания перспективного ПЗРК, который через несколько лет начнет поступать в Вооруженные силы на смену новейшего ПЗРК «Верба». Он будет защищен от тех лазерных помеховых комплексов, которые только еще поступают на вооружение передовых стран мира», — сказал Леонов.nnОтмечается, что ПЗРК «Верба» был принят на вооружение российской армии в 2015 году. Новый комплекс в 10 раз эффективнее преодолевает помехи.nРанееnстало известно, что новые средства войсковой ПВО в российской армии будут способны отражать все новейшие средства воздушного нападения, в том числе гиперзвуковые.


Четырехлетняя Жасмина Леонтьева, выжившая п

In [153]:
rnd = np.random.randint(1000, df.shape[0])
sentence = df.iloc[rnd, 6]

print(sentence)

cosine_scores, max_scores_idx = get_similar(embeddings, sentence, 3)
print("\n")
print(cosine_scores[0][max_scores_idx])
print("\n")

for t in sentences[max_scores_idx.tolist()]:
    print(t)
    print("\n")

ПоnданнымnМВД по Чеченской Республике, в митинге в поддержку мусульман Мьянмы, который прошел 4 сентября в Грозном, приняли участие 1,1 млн человек.n«В митинге, который состоялся сегодня в Грозном, приняли участие 1 100 000 человек. Общественно-массовое мероприятие прошло без каких-либо происшествий», — говорится в тексте официального сообщения.nОтмечается, что в ходе митинга не было допущено нарушений общественного порядка, жалоб на действия сотрудников внутренних органов республики.nРанееnсообщалосьn, что на митинге в Чечне Кадыров призвал остановить гуманитарную катастрофу в Мьянме.


tensor([0.5224, 0.5824, 0.5817])


В Тбилиси на национальном стадионе имени Бориса Пайчадзе в пятницу вечером прошел предвыборный митинг правящей в стране партии «Единое национальное движение». На трибунах стадиона, вмещающего 60 тысяч человек, все места были полностью заняты, еще около 10 тысяч человек расположились на беговых дорожках.nВыступая на митинге, президент Михаил Саакашвили заявил, что «это

#### Вывод:
Учитывая то, что похожие новости выбираются только из 1000 первых новостей - результат хороший. Тексты действительно похожие, когда cosine_scores выше 0.37 и 0.5. При значении ~0.25 схожесть хуже, замечается только в некоторых словах.